## Dask Scaling Study

We benchmark a suite of dask computations of three types:

1.  General task scheduling
2.  Multi-dimensional arrays
3.  Dataframes

In each case we consider computations of varying complexity including computations that are embarrassingly parallel, communication heavy, involve many small communications, or structured in non-trivial ways.

Additionally we consider tasks that are very fast (microseconds) so as to stress the scheduler early as well as tasks that take a more modest amount of time, usually 100ms each.

We perform the same computations in increasing sizes that scale linearly with the cluster size.  The cluster is composed of 2-core containers on Google compute engine ranging from 1 container to 256 containers (512 cores total).

### Benchmarking infrastructure

In [1]:
import math, time
from dask.distributed import Client, wait

In [2]:
from distributed.client import default_client
import pandas as pd

def run(func, client=None):
    client = client or default_client()
    client.restart()
    n = sum(client.ncores().values())
    coroutine = func(n)

    name, unit, numerator = next(coroutine)
    out = []
    while True:
        # time.sleep(1)
        start = time.time()
        try:
            next_name, next_unit, next_numerator = next(coroutine)
        except StopIteration:
            break
        finally:
            end = time.time()
            record = {'name': name, 
                      'duration': end - start, 
                      'unit': unit + '/s', 
                      'rate': numerator / (end - start), 
                      'n': n,
                      'collection': func.__name__}
            out.append(record)
        name = next_name
        unit = next_unit
        numerator = next_numerator
    return pd.DataFrame(out)

### Benchmarks

These functions include a variety of computations for tasks, arrays, and dataframes.

In [3]:
import operator
import time

def slowinc(x, delay=0.1):
    time.sleep(delay)
    return x + 1

def slowadd(x, y, delay=0.1):
    time.sleep(delay)
    return x + y

def slowsum(L, delay=0.1):
    time.sleep(delay)
    return sum(L)

def inc(x):
    return x + 1


def tasks(n):
    yield 'task map fast tasks', 'tasks', n * 200
    
    futures = client.map(inc, range(n * 200))
    wait(futures)
    
    yield 'task map 100ms tasks', 'tasks', n * 100

    futures = client.map(slowinc, range(100 * n))
    wait(futures)
        
    yield 'task map 1s tasks', 'tasks', n * 4

    futures = client.map(slowinc, range(4 * n), delay=1)
    wait(futures)

    yield 'tree reduction fast tasks', 'tasks', 2**7 * n
    
    from dask import delayed

    L = range(2**7 * n)
    while len(L) > 1:
        L = list(map(delayed(operator.add), L[0::2], L[1::2]))

    L[0].compute()
    
    yield 'tree reduction 100ms tasks', 'tasks', 2**6 * n * 2
    
    from dask import delayed

    L = range(2**6 * n)
    while len(L) > 1:
        L = list(map(delayed(slowadd), L[0::2], L[1::2]))

    L[0].compute()
    
    yield 'sequential', 'tasks', 100

    x = 1

    for i in range(100):
        x = delayed(inc)(x)
        
    x.compute()
    
    yield 'dynamic tree reduction fast tasks', 'tasks', 100 * n
    
    from dask.distributed import as_completed
    futures = client.map(inc, range(n * 100))
    
    pool = as_completed(futures)
    batches = pool.batches()
    
    while True:
        try:
            batch = next(batches)
            if len(batch) == 1:
                batch += next(batches)
        except StopIteration:
            break
        future = client.submit(sum, batch)
        pool.add(future)
        
    yield 'dynamic tree reduction 100ms tasks', 'tasks', 100 * n
    
    from dask.distributed import as_completed
    futures = client.map(slowinc, range(n * 20))
    
    pool = as_completed(futures)
    batches = pool.batches()
    
    while True:
        try:
            batch = next(batches)
            if len(batch) == 1:
                batch += next(batches)
        except StopIteration:
            break
        future = client.submit(slowsum, batch)
        pool.add(future)

        
    yield 'nearest neighbor fast tasks', 'tasks', 100 * n * 2
    
    L = range(100 * n)
    L = client.map(operator.add, L[:-1], L[1:])
    L = client.map(operator.add, L[:-1], L[1:])
    wait(L)
    
    yield 'nearest neighbor 100ms tasks', 'tasks', 20 * n * 2
    
    L = range(20 * n)
    L = client.map(slowadd, L[:-1], L[1:])
    L = client.map(slowadd, L[:-1], L[1:])
    wait(L)

In [4]:
def arrays(n):
    import dask.array as da
    N = int(5000 * math.sqrt(n))
    x = da.random.randint(0, 10000, size=(N, N), chunks=(2000, 2000))
    
    yield 'create random', 'MB', x.nbytes / 1e6
    
    x = x.persist()
    wait(x)
    
    yield 'blockwise 100ms tasks', 'MB', x.nbytes / 1e6
    
    y = x.map_blocks(slowinc, dtype=x.dtype).persist()
    wait(y)
    
    yield 'random access', 'bytes', 8
    
    x[1234, 4567].compute()
   
    yield 'reduction', 'MB', x.nbytes / 1e6
    
    x.std().compute()
    
    yield 'reduction along axis', 'MB', x.nbytes / 1e6
    
    x.std(axis=0).compute()
    
    yield 'elementwise computation', 'MB', x.nbytes / 1e6
    
    y = da.sin(x) ** 2 + da.cos(x) ** 2
    y = y.persist()
    wait(y)    
    
    yield 'rechunk small', 'MB', x.nbytes / 1e6
    
    y = x.rechunk((20000, 200)).persist()
    wait(y)
    
    yield 'rechunk large', 'MB', x.nbytes / 1e6
    
    y = y.rechunk((200, 20000)).persist()
    wait(y)
    
    yield 'transpose addition', 'MB', x.nbytes / 1e6
    y = x + x.T
    y = y.persist()
    wait(y)
    
    yield 'nearest neighbor fast tasks', 'MB', x.nbytes / 1e6
    
    y = x.map_overlap(inc, depth=1).persist()
    wait(y)   
        
    yield 'nearest neighbor 100ms tasks', 'MB', x.nbytes / 1e6
    
    y = x.map_overlap(slowinc, depth=1, delay=0.1).persist()
    wait(y)    

In [5]:
def dataframes(n):
    import dask.array as da
    import dask.dataframe as dd
    N = 2000000 * n
    
    x = da.random.randint(0, 10000, size=(N, 10), chunks=(1000000, 10))

    
    yield 'create random', 'MB', x.nbytes / 1e6
    
    df = dd.from_dask_array(x).persist()
    wait(df)
    
    yield 'blockwise 100ms tasks', 'MB', x.nbytes / 1e6
    
    wait(df.map_partitions(slowinc, meta=df).persist())
    
    yield 'arithmetic', 'MB', x.nbytes / 1e6
    
    y = (df[0] + 1 + 2 + 3 + 4 + 5 + 6 + 7 + 8 + 9 + 10).persist()
    wait(y)
    
    yield 'random access', 'bytes', 8
    
    df.loc[123456].compute()
    
    yield 'dataframe reduction', 'MB', x.nbytes / 1e6
    
    df.std().compute()
    
    yield 'series reduction', 'MB', x.nbytes / 1e6 / 10
    
    df[3].std().compute()
    
    yield 'groupby reduction', 'MB', x.nbytes / 1e6
    
    df.groupby(0)[1].mean().compute()
    
    yield 'groupby apply (full shuffle)', 'MB', x.nbytes / 1e6
    
    df.groupby(0).apply(len).compute()
    
    yield 'set index (full shuffle)', 'MB', x.nbytes / 1e6
    
    wait(df.set_index(1).persist())
    
    yield 'rolling aggregations', 'MB', x.nbytes / 1e6
    
    wait(df.rolling(5).mean().persist())

### Connect to cluster

We connect to a scheduler on GCE.  We start with a single 2-core worker.

In [7]:
from dask.distributed import Client
client = Client(host='ec2-18-234-108-14.compute-1.amazonaws.com:8786')
client

Client Scheduler: tcp://dask-scheduler:8786 Dashboard: http://dask-scheduler:8787,Cluster Workers: 1 Cores: 2 Memory: 6.77 GB


In [9]:
L = []

### Scale cluster and run computations

We use the [dask-kubernetes](https://github.com/martindurant/dask-kubernetes) command line to scale our cluster up and down manually and run the comptuations above multiple times for each scale.  We save these results to Google Cloud Storage.  The outputs are publicly available here:

-  Raw: https://storage.googleapis.com/dask-data/scaling-data-raw.csv
-  Median: https://storage.googleapis.com/dask-data/scaling-data.csv

In [98]:
client

Client Scheduler: tcp://dask-scheduler:8786 Dashboard: http://dask-scheduler:8787,Cluster Workers: 294 Cores: 588 Memory: 1991.50 GB


In [ ]:
%%time
for i in range(3):
    for func in [tasks, arrays, dataframes]:
        print(i, func.__name__)
        df = run(func, client=client)
        L.append(df)

0 tasks
0 arrays
0 dataframes


/work/miniconda/lib/python3.6/site-packages/ipykernel_launcher.py:41: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result


1 tasks
1 arrays
1 dataframes
2 tasks
2 arrays
2 dataframes
CPU times: user 9min 1s, sys: 3.69 s, total: 9min 5s
Wall time: 25min 10s


In [88]:
ddf = pd.concat(L)

In [89]:
df = ddf.groupby(['collection', 'name', 'n', 'unit']).median()
df

duration          rate
collection name                         n   unit                            
arrays     blockwise 100ms tasks        2   MB/s      0.930781    429.738441
                                        4   MB/s      0.847067    944.435197
                                        8   MB/s      1.053046   1519.373375
                                        16  MB/s      0.967746   3306.651083
                                        32  MB/s      1.113454   5747.768011
                                        64  MB/s      0.967302  13232.684357
                                        128 MB/s      1.289514  19852.058765
                                        256 MB/s      1.730550  29585.964835
                                        512 MB/s      6.030538  16980.218490
           create random                2   MB/s      0.584174    684.713900
                                        4   MB/s      0.655029   1221.319419
                                        8   MB/s      0.721304   2218.161714
                                        16  MB/s      0.664351   4816.727556
                                        32  MB/s      0.718007   8913.387113
                                        64  MB/s      0.738002  17344.136192
                                        128 MB/s      1.200436  21325.182436
                                        256 MB/s      2.564872  19962.007739
                                        512 MB/s      1.880598  54450.695152
           elementwise computation      2   MB/s      2.708794    147.664339
                                        4   MB/s      2.908163    275.087715
                                        8   MB/s      3.580060    446.911371
                                        16  MB/s      3.964876    807.087044
                                        32  MB/s      5.441955   1176.025319
                                        64  MB/s      7.383476   1733.600921
                                        128 MB/s     13.116995   1951.629145
                                        256 MB/s     21.965428   2330.935700
                                        512 MB/s     42.592467   2404.177413
           nearest neighbor 100ms tasks 2   MB/s      1.236899    323.383141
                                        4   MB/s      1.160823    689.166299
                                        8   MB/s      1.509198   1060.145264
...                                                        ...           ...
tasks      task map 1s tasks            128 tasks/s   5.590187     91.589068
                                        256 tasks/s   5.644582    181.412908
                                        512 tasks/s   7.549867    271.263068
           task map fast tasks          2   tasks/s   0.221133   1808.870110
                                        4   tasks/s   0.241904   3307.103271
                                        8   tasks/s   0.486054   3291.817309
                                        16  tasks/s   0.800579   3997.107929
                                        32  tasks/s   1.796089   3563.297941
                                        64  tasks/s   3.346050   3825.405776
                                        128 tasks/s   6.896783   3711.875589
                                        256 tasks/s  14.315529   3576.535636
                                        512 tasks/s  29.077214   3521.657944
           tree reduction 100ms tasks   2   tasks/s   6.856049     37.339291
                                        4   tasks/s   7.081078     72.305377
                                        8   tasks/s   7.219373    141.840563
                                        16  tasks/s   7.429270    275.666383
                                        32  tasks/s   7.793831    525.543849
                                        64  tasks/s   8.285013    988.773376
                                        128 tasks/s   8.970450   1826.441178
                                        256 tasks/

In [90]:
df.to_csv('scaling-data.csv')

In [91]:
import gcsfs
gcs = gcsfs.GCSFileSystem(token='cloud')
gcs.put('scaling-data.csv', 'dask-data/scaling-data.csv')

In [92]:
ddf.to_csv('scaling-data-raw.csv')
gcs.put('scaling-data-raw.csv', 'dask-data/scaling-data-raw.csv')